In [12]:
import json

with open('./Tweetsum_Json/final_test_tweetsum.jsonl', 'r') as json_file:
    test_data = list(json_file)


In [48]:
import ast
ast.literal_eval(test_data[0])['annotations'][0]['extractive']
# ast.literal_eval(test_data[0])['annotations'][0]['abstractive']

[{'tweet_id': 2263653, 'sentence_offset': '[0, 80]'},
 {'tweet_id': 2263654, 'sentence_offset': '[15, 114]'},
 {'tweet_id': 2263656, 'sentence_offset': '[43, 132]'},
 {'tweet_id': 2263658, 'sentence_offset': '[46, 108]'}]

In [16]:
import ast
import pandas as pd

twcs = pd.read_csv('./twcs.csv')

twcs_dict = {}

for i in range(len(twcs)):

    twcs_dict[twcs.iloc[i]['tweet_id']] = [twcs.iloc[i]['author_id'], twcs.iloc[i]['text']]

    if i%1000 == 0:
        print(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [27]:
twcs_dict_new = {}

for key in list(twcs_dict.keys()):
    twcs_dict_new[int(key)] = twcs_dict[key]

In [28]:
len(twcs_dict_new)

2811774

In [26]:
type(ast.literal_eval(test_data[0])["tweet_ids_sentence_offset"][0]['tweet_id'])

int

In [29]:
len(twcs_dict)
import json

with open('./twcs_text.json', 'w') as f:
    json.dump(twcs_dict_new, f)
    

In [81]:
test= {'conversation_id': '1fbfbb7710a2c53a781ba3dd7f7c08c0', 'tweet_ids_sentence_offset': [{'tweet_id': 2256664, 'sentence_offsets': ['[0, 27]']}, {'tweet_id': 2256661, 'sentence_offsets': ['[0, 39]', '[40, 123]', '[124, 130]']}, {'tweet_id': 2256662, 'sentence_offsets': ['[0, 73]', '[74, 114]', '[115, 125]']}, {'tweet_id': 2256660, 'sentence_offsets': ['[0, 59]', '[60, 113]']}, {'tweet_id': 2256663, 'sentence_offsets': ['[0, 119]']}, {'tweet_id': 2256659, 'sentence_offsets': ['[0, 29]', '[30, 89]', '[90, 96]']}, {'tweet_id': 2256658, 'sentence_offsets': ['[0, 74]']}, {'tweet_id': 2256655, 'sentence_offsets': ['[0, 68]', '[69, 123]', '[124, 130]']}, {'tweet_id': 2256657, 'sentence_offsets': ['[0, 19]']}, {'tweet_id': 2256656, 'sentence_offsets': ['[0, 33]']}], 'annotations': [{'extractive': '', 'abstractive': ['Customer says he is not satisfied with the company service.', 'Agent asks to send the address so that he can double check which service is provided to the scott.']}, {'extractive': [{'tweet_id': 2256662, 'sentence_offset': '[74, 114]'}, {'tweet_id': 2256663, 'sentence_offset': '[0, 119]'}, {'tweet_id': 2256655, 'sentence_offset': '[0, 68]'}, {'tweet_id': 2256655, 'sentence_offset': '[69, 123]'}], 'abstractive': ['Customer is complaining about the slowness in the available service at his location.', 'Agent is requesting for the address in order to investigate further  via Direct message.']}]}

In [84]:
test['annotations'][0]

{'conversation_id': '1fbfbb7710a2c53a781ba3dd7f7c08c0',
 'tweet_ids_sentence_offset': [{'tweet_id': 2256664,
   'sentence_offsets': ['[0, 27]']},
  {'tweet_id': 2256661,
   'sentence_offsets': ['[0, 39]', '[40, 123]', '[124, 130]']},
  {'tweet_id': 2256662,
   'sentence_offsets': ['[0, 73]', '[74, 114]', '[115, 125]']},
  {'tweet_id': 2256660, 'sentence_offsets': ['[0, 59]', '[60, 113]']},
  {'tweet_id': 2256663, 'sentence_offsets': ['[0, 119]']},
  {'tweet_id': 2256659,
   'sentence_offsets': ['[0, 29]', '[30, 89]', '[90, 96]']},
  {'tweet_id': 2256658, 'sentence_offsets': ['[0, 74]']},
  {'tweet_id': 2256655,
   'sentence_offsets': ['[0, 68]', '[69, 123]', '[124, 130]']},
  {'tweet_id': 2256657, 'sentence_offsets': ['[0, 19]']},
  {'tweet_id': 2256656, 'sentence_offsets': ['[0, 33]']}],
 'annotations': [{'extractive': '',
   'abstractive': ['Customer says he is not satisfied with the company service.',
    'Agent asks to send the address so that he can double check which service is p

In [73]:
for t in test_data:
    try:
        d = ast.literal_eval(t)
        print(d)
        break
    except:
        print(t)
        break

{'conversation_id': 'bbde6d8ec7c39c4551da1ff6024f997b', 'tweet_ids_sentence_offset': [{'tweet_id': 2263653, 'sentence_offsets': ['[0, 80]', '[82, 95]']}, {'tweet_id': 2263651, 'sentence_offsets': ['[0, 43]', '[44, 68]', '[69, 134]']}, {'tweet_id': 2263652, 'sentence_offsets': ['[0, 57]']}, {'tweet_id': 2263654, 'sentence_offsets': ['[0, 14]', '[15, 114]']}, {'tweet_id': 2263655, 'sentence_offsets': ['[0, 24]', '[25, 76]']}, {'tweet_id': 2263656, 'sentence_offsets': ['[0, 42]', '[43, 132]']}, {'tweet_id': 2263657, 'sentence_offsets': ['[0, 67]', '[68, 118]', '[119, 163]', '[164, 177]']}, {'tweet_id': 2263658, 'sentence_offsets': ['[0, 16]', '[17, 45]', '[46, 108]', '[109, 110]']}], 'annotations': [{'extractive': [{'tweet_id': 2263653, 'sentence_offset': '[0, 80]'}, {'tweet_id': 2263654, 'sentence_offset': '[15, 114]'}, {'tweet_id': 2263656, 'sentence_offset': '[43, 132]'}, {'tweet_id': 2263658, 'sentence_offset': '[46, 108]'}], 'abstractive': ['Customer is complaining that the watchlist

In [ ]:
import json

test_data_2 = [json.loads(jline) for jline in jsonl_content.splitlines()]

In [91]:
import json
with open('./Tweetsum_Json/final_test_tweetsum.jsonl') as f:
    test_data2 = [json.loads(line) for line in f]

In [92]:
test_data2[0]

{'conversation_id': 'bbde6d8ec7c39c4551da1ff6024f997b',
 'tweet_ids_sentence_offset': [{'tweet_id': 2263653,
   'sentence_offsets': ['[0, 80]', '[82, 95]']},
  {'tweet_id': 2263651,
   'sentence_offsets': ['[0, 43]', '[44, 68]', '[69, 134]']},
  {'tweet_id': 2263652, 'sentence_offsets': ['[0, 57]']},
  {'tweet_id': 2263654, 'sentence_offsets': ['[0, 14]', '[15, 114]']},
  {'tweet_id': 2263655, 'sentence_offsets': ['[0, 24]', '[25, 76]']},
  {'tweet_id': 2263656, 'sentence_offsets': ['[0, 42]', '[43, 132]']},
  {'tweet_id': 2263657,
   'sentence_offsets': ['[0, 67]', '[68, 118]', '[119, 163]', '[164, 177]']},
  {'tweet_id': 2263658,
   'sentence_offsets': ['[0, 16]', '[17, 45]', '[46, 108]', '[109, 110]']}],
 'annotations': [{'extractive': [{'tweet_id': 2263653,
     'sentence_offset': '[0, 80]'},
    {'tweet_id': 2263654, 'sentence_offset': '[15, 114]'},
    {'tweet_id': 2263656, 'sentence_offset': '[43, 132]'},
    {'tweet_id': 2263658, 'sentence_offset': '[46, 108]'}],
   'abstractiv

In [108]:
parsed = []

for t in test_data2:
    # try:
    #     t_dict = ast.literal_eval(t)

    # except:
    #     # print(t)
    #     r = t.replace("\"", "\'")
    #     r = r.replace('null', '\'\'')
    #     t_dict = ast.literal_eval(r)

    # print(t_dict)

    anno = t['annotations']

    for a in anno:
        
        abst = a['abstractive']
        tweets = a['extractive']

        if tweets is not None:

            dialog = []
            for t in tweets:
                dialog.append(twcs_dict[t['tweet_id']][0] + ": " + twcs_dict[t['tweet_id']][1] + "\n")
        
            parsed.append(["".join(dialog), " ".join(abst)])


parsed


[["658975: @hulu_support My watchlist is not updating with new episodes (past couple days).  Any idea why?\nhulu_support: @658975 Sorry! 😔 We assure you that our team is working hard to investigate, and we hope to have a fix ready soon!\nhulu_support: @658975 We definitely understand, Norlene. For now, we recommend checking the show page for these shows as the new eps will be there\nhulu_support: @658975 Awesome! That's what we love to hear. If you happen to need anything else, we'll be here to support! 💚\n",
  'Customer is complaining that the watchlist is not updated with new episodes from past two days. Agent informed that the team is working hard to investigate to show new episodes on page.'],
 ["658975: @hulu_support My watchlist is not updating with new episodes (past couple days).  Any idea why?\nhulu_support: @658975 Apologies for the trouble, Norlene! We're looking into this. In the meantime, try navigating to the season / episode manually.\nhulu_support: @658975 Sorry! 😔 We a

In [109]:
len(parsed)

test_parsed = pd.DataFrame(parsed, columns=['dialog', 'summary'])

In [110]:
print(test_parsed.iloc[0]['dialog'])
print('---')
print(test_parsed.iloc[0]['summary'])

658975: @hulu_support My watchlist is not updating with new episodes (past couple days).  Any idea why?
hulu_support: @658975 Sorry! 😔 We assure you that our team is working hard to investigate, and we hope to have a fix ready soon!
hulu_support: @658975 We definitely understand, Norlene. For now, we recommend checking the show page for these shows as the new eps will be there
hulu_support: @658975 Awesome! That's what we love to hear. If you happen to need anything else, we'll be here to support! 💚

---
Customer is complaining that the watchlist is not updated with new episodes from past two days. Agent informed that the team is working hard to investigate to show new episodes on page.


In [111]:
test_parsed.to_csv('./twcs_test.csv')

In [116]:
def run_summarizer(input):
    output_list = []

    from transformers import pipeline
    summarizer = pipeline("summarization", model="linydub/bart-large-samsum")

    for i in range(len(input)):
        output_list.append(summarizer(input[i]))
        print(i)

    return output_list

In [117]:
test_result = run_summarizer(test_parsed['dialog'].tolist())

test_parsed['predicted'] = test_result

test_parsed.to_csv('./twcs_test_predicted.csv')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82


Your max_length is set to 62, but you input_length is only 60. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139


Your max_length is set to 62, but you input_length is only 40. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


140
141
142
143
144
145
146
147
148


...

149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194


Your max_length is set to 62, but you input_length is only 56. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266


Your max_length is set to 62, but you input_length is only 60. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310


In [69]:
import ast

twcs_test = []

for i in range(len(test_data)):
    try:
        current = ast.literal_eval(test_data[i].strip())['annotations'][0]
    except:
        # print(ast.literal_eval(test_data[i].strip()))
        print(eval(test_data[i]))
        print('----')
        break
        # print(test_data[i])
    # print(current)
    tweets = current['extractive']

    abst = ' '.join(current['abstractive'])

    ast.literal_eval(test_data[0])['annotations'][0]['extractive']
    ast.literal_eval(test_data[0])['annotations'][0]['abstractive']
 
    dialog = []
    for t in tweets:
        dialog.append(twcs_dict[t['tweet_id']][0] + ": " + twcs_dict[t['tweet_id']][1] + "\n")

    twcs_test.append(['\n'.join(dialog), abst])
        


NameError: name 'null' is not defined

In [12]:
import pandas as pd

tweetsum = pd.read_csv('./tweeter_cs/sample.csv')

In [16]:
for i in range(10):
    print(tweetsum.iloc[i]['text'])

@AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡
@105835 Your business means a lot to us. Please DM your name, zip code and additional details about your concern. ^RR https://t.co/znUu1VJn9r
@76328 I really hope you all change but I'm sure you won't! Because you don't have to!
@105836 LiveChat is online at the moment - https://t.co/SY94VtU8Kq or contact 03331 031 031 option 1, 4, 3 (Leave a message) to request a call back
@VirginTrains see attached error message. I've tried leaving a voicemail several times in the past week https://t.co/NxVZjlYx1k
@105836 Have you tried from another device, Miriam ^MM
@VirginTrains yep, I've tried laptop too several times over the past week and again today. I've tried different browsers too
@105836 It's working OK from here, Miriam. Does this link help https://t.co/0m2mpH15eh ? ^MM
@VirginTrains I still haven't heard &amp; the number I'm directed to by phone is a dead end &amp; the live chat 

In [11]:
from transformers import pipeline
summarizer = pipeline("summarization", model="linydub/bart-large-samsum")

input_text = '''
    Henry: Hey, is Nate coming over to watch the movie tonight?
    Kevin: Yea, he said he'll be arriving a bit later at around 7 since he gets off of work at 6. Have you taken out the garbage yet?
    Henry: Oh I forgot. I'll do that once I'm finished with my assignment for my math class.
    Kevin: Yea, you should take it out as soon as possible. And also, Nate is bringing his girlfriend.
    Henry: Nice, I'm really looking forward to seeing them again.
'''
summarizer(input_text)

c:\Users\nahyun\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 1.63k/1.63k [00:00<00:00, 1.63MB/s]
c:\Users\nahyun\anaconda3\envs\torch\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nahyun\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an ad

[{'summary_text': 'Nate and his girlfriend are coming over to watch the movie tonight. Kevin reminds Henry to take out the garbage.'}]